## Joins!

- More likely, you will have to stitch data from a few different sources together in order to create the data representation needed to solve the problem at hand.

- Joins are common within the data pipeline as a solution to combining data. 
- These workflows fall under the umbrella of the ETL and can be used whenever you need to strategically combine and transform multiple sources of data into a single consolidated view that can be used to answer more targeted problems.
- For example, say we were tasked with creating a job that generates the current available occupancy data for our coffee shops. For the sake of the exercise, let’s say we already have a source of data that emits the number of occupied seats per coffee shop. 
- We can use this data to join with our coffee stores data to create a new view that we can query to find which store can seat a variable sized party.

In [1]:
# Import required modules
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql import functions as f
from pyspark.sql import *
from pyspark.sql.types import *
# Delta is a storage layer for data lakes
from delta.tables import * 
# DeltaTable is the main class for Delta tables
from delta.tables import DeltaTable 

# Initialize SparkSession
# Create a SparkSession and set the extraClassPath configuration
spark = SparkSession.builder.master("local[1]") \
    .appName("StoreOccupants") \
    .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/*") \
    .getOrCreate()


# Define the schema for the Store class
store_schema = StructType([
    StructField("name", StringType(), True),
    StructField("capacity", IntegerType(), True),
    StructField("opens", IntegerType(), True),
    StructField("closes", IntegerType(), True)
])

# Create a list of Row objects
stores = [
    Row("a", 24, 8, 20),
    Row("b", 36, 7, 21),
    Row("c", 18, 5, 23)
]

# Create a PySpark DataFrame from the Row objects and schema
stores_sdf = spark.createDataFrame(stores, store_schema)
stores_sdf.show()

# Define the schema for the Occupants
schema = StructType([
  StructField("storename", StringType(), True),
  StructField("occupants", IntegerType(), True)
])

# Create a PySpark DataFrame from the occupants sequence
occupants_sdf = spark.createDataFrame([
  ("a", 8),
  ("b", 20),
  ("c", 16),
  ("d", 55),
  ("e", 8)
], schema)

# Show the DataFrame
occupants_sdf.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----+--------+-----+------+
|name|capacity|opens|closes|
+----+--------+-----+------+
|   a|      24|    8|    20|
|   b|      36|    7|    21|
|   c|      18|    5|    23|
+----+--------+-----+------+

+---------+---------+
|storename|occupants|
+---------+---------+
|        a|        8|
|        b|       20|
|        c|       16|
|        d|       55|
|        e|        8|
+---------+---------+



### Create SQL view 

In [2]:
# Register the stores and store_occupants DataFrames as temporary views
stores_sdf.createOrReplaceTempView("stores")
occupants_sdf.createOrReplaceTempView("store_occupants")

## Inner Join

- The inner join is the simplest to understand and just so happens to also be the default join operation in Spark (given this is usually how people want to join data). 
- The inner join works by selecting only the rows that meet the join selection criteria across both sides of the data being joined.
- Inner joins simply ignore all rows that don’t have a matching join condition. Next up, let’s look at the right join.

In [3]:
# Execute the SparkSQL query
innerjoin_df = spark.sql("SELECT * FROM stores INNER JOIN store_occupants ON stores.name = store_occupants.storename")

# Show the joined DataFrame
innerjoin_df.show()

+----+--------+-----+------+---------+---------+
|name|capacity|opens|closes|storename|occupants|
+----+--------+-----+------+---------+---------+
|   a|      24|    8|    20|        a|        8|
|   b|      36|    7|    21|        b|       20|
|   c|      18|    5|    23|        c|       16|
+----+--------+-----+------+---------+---------+



- The result of our join operation is a new DataFrame that combines all the columns of our two data sources where the join criteria is met. 
- In this case, that’s where there is a matching store name across both data sources.

## Right Join

- The right join, or right outer join, returns all rows from the right-side data source explicitly joining all rows where the selection criteria is met with the left side of the data. 
- When and the data doesn’t match, it will insert null values instead.

In [4]:
# Execute the SparkSQL query
rightjoin_df = spark.sql("SELECT stores.*, store_occupants.occupants FROM stores RIGHT JOIN store_occupants ON stores.name = store_occupants.storename")

# Show the joined DataFrame
rightjoin_df.show()

+----+--------+-----+------+---------+
|name|capacity|opens|closes|occupants|
+----+--------+-----+------+---------+
|   a|      24|    8|    20|        8|
|   b|      36|    7|    21|       20|
|   c|      18|    5|    23|       16|
|null|    null| null|  null|       55|
|null|    null| null|  null|        8|
+----+--------+-----+------+---------+



**DataframeAPI**

In [5]:
# `stores_sdf` is our stores data DataFrame
right_joined = stores_sdf.join(occupants_sdf, stores_sdf["name"] == occupants_sdf["storename"], "right")
right_joined.show()

+----+--------+-----+------+---------+---------+
|name|capacity|opens|closes|storename|occupants|
+----+--------+-----+------+---------+---------+
|   a|      24|    8|    20|        a|        8|
|   b|      36|    7|    21|        b|       20|
|   c|      18|    5|    23|        c|       16|
|null|    null| null|  null|        d|       55|
|null|    null| null|  null|        e|        8|
+----+--------+-----+------+---------+---------+



- In a right join, all rows from the right table (store_occupants in this case) are included in the result, along with matching rows from the left table (stores in this case).
- If there are no matching rows in the left table for a given row in the right table, then the result will contain NULL values for the columns of the left table.

**Therefore, in this query, the store_occupants table is on the right side of the join, and the stores table is on the left side.**